## Tax Deduction

Aggregating hours worked from home (WFH) for the Financial Year 2023-24

In [1]:
import PyPDF2
import os
import pandas as pd
import time

In [2]:
# payslips for the mentioned FY are put in this folder
directory_path = './FY23-24'
# fetch their names for computation
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and not f.startswith('.DS_Store')]
# sort the list by name
filenames.sort()

In [3]:
# intitalise an empty list to store tuples (date, code, hours, income) for each claim
claims_li = []
# for tracking time
time_start = time.time()
# iterate over all payslips
for file in filenames:
    # read data from the PDF
    reader = PyPDF2.PdfReader('FY23-24/'+str(file))
    # extract data from reader object
    data = reader.pages[0].extract_text().split("\n")
    # iterate through each line where "i" is line number and "j" is the content within that line 
    for i,j in enumerate(data):
        # select only the relevant part of the payslip i.e. the claims part
        if "SALARY AND ALLOWANCES" in j:
            start = i
        if "TOTAL SALARY AND  ALLOWANCES" in j:
            end = i
    # traverse through all the claims
    for i in reader.pages[0].extract_text().split("\n")[start+2:end]:
        # create a tuple with date, wage type/code, hours, total amount and add it to the list
        claims_li.append((i.split()[1],int(i.split()[2]), float(i.split()[-4]), float(i.split()[-2])))
time.time() - time_start

0.44251585006713867

In [4]:
# convert list to pandas df
df = pd.DataFrame(claims_li, columns=['date', 'code', 'hours', 'amount'])

In [5]:
# dictionary to store the claim code and its description
code_desc = {2009: 'MonashConnect', 2088: 'ORAA', 2093:'RA', 2091:'Marking', 2269: 'WorkshopSupport', 2271:'AppliedClass'}

In [6]:
# add description to the df using dictionary
df['description'] = df['code'].map(code_desc)

In [7]:
df

,date,code,hours,amount,description
0,17/07,2088,1.00,52.94,ORAA
1,24/07,2088,2.50,132.35,ORAA
2,27/07,2088,1.00,52.94,ORAA
3,31/07,2088,2.50,132.35,ORAA
4,03/08,2088,1.00,52.94,ORAA
...,...,...,...,...,...
139,03/06,2091,4.00,218.12,Marking
140,04/06,2091,4.00,218.12,Marking
141,05/06,2091,4.00,218.12,Marking
142,06/06,2091,4.00,218.12,Marking


In [8]:
# exclude claims that cannot be used for wfh deductions
new_df = df[~df['code'].isin([2269,2271])]
sum(new_df['hours'])

217.49

In [37]:
new_df['description'].unique()

array(['ORAA', 'Marking'], dtype=object)

In [27]:
new_df.to_csv('summary_FY23-24.csv')